# Modelo NLP: Entrenamiento

In [ ]:
import datetime

In [ ]:
import sagemaker
import json
import boto3

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
prefix = 'module_4/part_05'

#### Subimos los datos a s3

In [ ]:
train_channel = prefix + '/train'
sess.upload_data(path='dbpedia_csv/dbpedia.train', bucket=bucket, key_prefix=train_channel)

In [ ]:
validation_channel = prefix + '/validation'
sess.upload_data(path='dbpedia_csv/dbpedia.validation', bucket=bucket, key_prefix=validation_channel)

#### Modelo

- Usaremos el modelo https://docs.aws.amazon.com/es_es/sagemaker/latest/dg/blazingtext.html

- Amazon SageMaker BlazingText proporciona implementaciones altamente optimizadas de los algoritmos de Word2VEC y de clasificación de texto. El algoritmo Word2vec es útil para muchas tareas de procesamiento de lenguaje natural (NLP) posteriores, como por ejemplo, análisis del sentimiento de reconocimiento de entidad nombrada, traducción automática, etc. La clasificación de texto es una tarea importante para las aplicaciones que realizan búsquedas web, recuperación de información, funciones de clasificación y clasificación de documentos.

- El algoritmo Word2vec asigna palabras a vectores distribuidos de alta calidad. La representación vectorial resultante de una palabra se denomina una incrustación de palabra. Las palabras similares desde el punto de vista semántico corresponden a vectores que se acercan entre sí. De esta forma, las incrustaciones de palabras capturan relaciones semánticas entre palabras. 

In [ ]:
image=sagemaker.image_uris.retrieve(framework='blazingtext', 
                                    region=region, 
                                    version='1')
print(image)

In [ ]:
s3_output_location = f's3://{bucket}/{prefix}/output'

estimator = sagemaker.estimator.Estimator(
        image,
        role,
        instance_count=1,
        instance_type='ml.c5.2xlarge',
        volume_size=30,
        max_run=360000,
        input_mode='File',
        enable_sagemaker_metrics=True,
        output_path=s3_output_location,
        hyperparameters={
            'mode': 'supervised',
            'epochs': 20,
            'min_count': 2,
            'learning_rate': 0.05,
            'vector_dim': 10,
            'early_stopping': True,
            'patience': 4,
            'min_epochs': 5,
            'word_ngrams': 2,
        },
)

#### Entrenamiento

In [ ]:
s3_train_data = f's3://{bucket}/{train_channel}'
s3_validation_data = f's3://{bucket}/{validation_channel}'
print(s3_train_data)
print(s3_validation_data)

data_channels = {'train': s3_train_data, 
                 'validation': s3_validation_data}

now = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
jobname = f'dbpedia-blazingtext-{now}'

In [ ]:
estimator.fit(inputs=data_channels,
              job_name=jobname,
              logs=True)

Once the training job finishes, you can access the trained model in estimator.model_data
which can later be used for hosting and inferencing either in the cloud, which is a topic we will explore in depth in the next chapter, or on a computer with the fastText program. 
You can access the model with the following code block:

In [ ]:
estimator.model_data

In [ ]:
!aws s3 cp {estimator.model_data} ./dbpedia_csv/

In [ ]:
%%sh
cd dbpedia_csv/
tar -zxf model.tar.gz